In [ ]:
# %pip install -Uq gradio
import gradio as gr

import random

In [ ]:
n = 0
while n < 5:
    print(n)
    n += 1

In [ ]:
guess = None
answer = random.randint(1, 99)

while guess != answer:
    guess = input("猜一個數字：")
    guess = int(guess)

    if guess < answer:
        print("太小了！")
    elif guess > answer:
        print("太大了！")

print("答對了，遊戲結束！")

In [ ]:
# 基本猜數字遊戲
guess = None
answer = random.randint(1, 99)

while guess != answer:
    guess = input("🤔 從 0 到 100 裡面猜一個數字：")
    guess = int(guess)

    if guess < answer:
        print(f"⬆️ {guess} 太小了！")
    elif guess > answer:
        print(f"⬇️ {guess} 太大了！")

print(f"🎉 {guess} 答對了！")
print("🎮 遊戲結束！")

In [ ]:
# 會更新範圍的猜數字遊戲
answer = random.randint(1, 99)
guess = None
lower = 0
upper = 100

while guess != answer:
    guess = input(f"🤔 從 {lower} 到 {upper} 裡面猜一個數字：")
    guess = int(guess)

    if guess < answer:
        print(f"⬆️ {guess} 太小了！")
        lower = max(lower, guess)
    elif guess > answer:
        print(f"⬇️ {guess} 太大了！")
        upper = min(upper, guess)

print(f"🎉 {guess} 答對了！")
print("🎮 遊戲結束！")

In [ ]:
# 嘗試用全域變數來記錄範圍

with gr.Blocks() as app:
    answer = random.randint(1, 99)
    upper = 100
    lower = 0

    guess = gr.Number(label="🤔 猜數字")
    hint = gr.Textbox("🎮 從 0 到 100 裡面猜一個數字", label="提示")

    def check_guess(guess):
        global upper, lower, answer

        if guess == answer:
            msg = f"🎉 答對了，答案是 {answer}！再來玩一次吧～"
            answer = random.randint(1, 99)
            upper = 100
            lower = 0
            return msg

        if guess < answer:
            lower = guess
            return f"⬆️ 太小了，{lower} 到 {upper} 猜一個數字"

        if guess > answer:
            upper = guess
            return f"⬇️ 太大了，{lower} 到 {upper} 猜一個數字"

    guess.submit(check_guess, guess, hint, show_progress=False)

app.launch()

In [ ]:
# 透過 gr.State 來記錄範圍，避免多重使用者的情況下會發生的衝突

with gr.Blocks() as app:
    answer = random.randint(1, 99)
    upper = gr.State(100)
    lower = gr.State(0)

    guess = gr.Number(label="🤔 猜數字")
    hint = gr.Textbox("🎮 從 0 到 100 裡面猜一個數字", label="提示")

    def check_guess(guess, answer, upper, lower):
        if guess == answer:
            msg = f"🎉 答對了，答案是 {answer}！再來玩一次吧～"
            answer = random.randint(1, 99)
            upper = 100
            lower = 0
            return msg, answer, upper, lower

        if guess < answer:
            lower = guess
            msg = f"⬆️ 太小了，{lower} 到 {upper} 猜一個數字"
            return msg, answer, upper, lower

        if guess > answer:
            upper = guess
            msg = f"⬇️ 太大了，{lower} 到 {upper} 猜一個數字"
            return msg, answer, upper, lower

    guess.submit(
        check_guess,
        [guess, answer, upper, lower],
        [hint, answer, upper, lower],
        show_progress=False,
    )

app.launch()

In [ ]:
# 透過 gr.State(None) 的技巧，來避免大家的第一個正解都一樣的問題

with gr.Blocks() as app:
    answer = gr.State(None)
    upper = gr.State(100)
    lower = gr.State(0)

    guess = gr.Number(label="🤔 猜數字")
    hint = gr.Textbox("🎮 從 0 到 100 裡面猜一個數字", label="提示")

    def check_guess(guess, answer, upper, lower):
        if answer is None:
            answer = random.randint(1, 99)

        if guess == answer:
            msg = f"🎉 答對了，答案是 {answer}！再來玩一次吧～"
            answer = random.randint(1, 99)
            upper = 100
            lower = 0
            return msg, answer, upper, lower

        if guess < answer:
            lower = max(lower, guess)
            msg = f"⬆️ 太小了，{lower} 到 {upper} 猜一個數字"
            return msg, answer, upper, lower

        if guess > answer:
            upper = min(upper, guess)
            msg = f"⬇️ 太大了，{lower} 到 {upper} 猜一個數字"
            return msg, answer, upper, lower

    guess.submit(
        check_guess,
        [guess, answer, upper, lower],
        [hint, answer, upper, lower],
        show_progress=False,
    )

app.launch()

In [1]:
# 加上花花綠綠的小裝飾
import gradio as gr


css = """
/* 適用於 Chrome、Safari、Edge */
input[type="number"]::-webkit-outer-spin-button,
input[type="number"]::-webkit-inner-spin-button {
    -webkit-appearance: none;
    margin: 0;
}

/* 適用於 Firefox */
input[type="number"] {
    -moz-appearance: textfield;
}

/* 適用於大多數瀏覽器 */
* {
  overflow: auto;
  scrollbar-width: none; /* Firefox */
  -ms-overflow-style: none;  /* IE 10+ */
}

*::-webkit-scrollbar {
  display: none; /* Chrome、Safari */
}

html, body {
  overflow: auto;
}
"""

font = gr.themes.GoogleFont("LXGW WenKai TC")
theme = gr.themes.Ocean(
    primary_hue="emerald",
    secondary_hue="sky",
    neutral_hue="zinc",
    font=font,
    text_size=gr.themes.sizes.text_md,
    radius_size=gr.themes.sizes.radius_md,
    spacing_size=gr.themes.sizes.spacing_md,
)

with gr.Blocks(theme=theme, css=css, title="單手猜數字小精靈") as app:
    answer = gr.State(None)
    upper = gr.State(100)
    lower = gr.State(0)

    gr.Markdown("## 🧙‍♂️ 🔮 單手猜數字小精靈 ✨ 🔍")
    gr.Markdown("### 👉 過彎不減速的摔車王 🚴 可以猜對這個數字嗎 🤕")
    with gr.Group():
        guess = gr.Number(label="🤔 數字", value=50)
        hint = gr.Textbox("🎮 從 0 到 100 裡面猜一個數字", label="💡 提示")
        with gr.Row():
            win_count = gr.Number(label="🏆 贏的次數", interactive=False)
            try_count = gr.Number(label="👀 猜的次數", interactive=False)
            win_rate = gr.Textbox("0.00%", label="🎯 贏的比例", interactive=False)

    def check_guess(guess, answer, upper, lower, try_count, win_count, win_rate):
        if answer is None:
            answer = random.randint(1, 99)

        if guess == answer:
            win_count += 1
            lower = 0
            upper = 100
            answer = random.randint(1, 99)
            hint = f"🎉 答對了，答案是 {answer}！再來玩一次吧～"
        elif guess < answer:
            lower = guess
            hint = f"⬆️ 太小了，{lower} 到 {upper} 猜一個數字"
        elif guess > answer:
            upper = guess
            hint = f"⬇️ 太大了，{lower} 到 {upper} 猜一個數字"

        guess = (lower + upper) // 2
        try_count += 1
        win_rate = win_count / try_count
        win_rate = f"{win_rate:.2%}"

        return guess, hint, answer, upper, lower, try_count, win_count, win_rate

    guess.submit(
        check_guess,
        [guess, answer, upper, lower, try_count, win_count, win_rate],
        [guess, hint, answer, upper, lower, try_count, win_count, win_rate],
        show_progress=False,
    )

app.launch(favicon_path="icon-v3.png")

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [ ]:
# 展示不同的佈景主題

theme_list = [
    gr.themes.Base,
    gr.themes.Default,
    gr.themes.Origin,
    gr.themes.Glass,
    gr.themes.Monochrome,
    gr.themes.Soft,
    gr.themes.Citrus,
    gr.themes.Ocean,
]

for theme_cls in theme_list:
    theme = theme_cls(
        spacing_size=gr.themes.sizes.spacing_md,
        text_size=gr.themes.sizes.text_md,
        radius_size=gr.themes.sizes.radius_md,
    )

    with gr.Blocks(theme=theme) as app:
        gr.Markdown(f"# {theme_cls.__name__}")
        with gr.Tab("Tab 1"):
            with gr.Row():
                gr.Textbox(label="Textbox", placeholder="Placeholder", submit_btn=True)
                gr.Slider(label="Slider", minimum=0, maximum=100, value=50)

            with gr.Row():
                gr.CheckboxGroup(
                    label="Checkbox",
                    choices=["Option 1", "Option 2"],
                    value=["Option 1"],
                )
                gr.DateTime(label="DateTime", value="2025-04-02 21:00:00", interactive=True)

            with gr.Row():
                gr.ClearButton()
                gr.Button("Button", variant="primary")

        with gr.Tab("Tab 2"):
            pass

        with gr.Tab("Tab 3"):
            pass

        app.launch()